In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from pydantic import Field, BaseModel
from dotenv import load_dotenv
import os
import operator
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
def get_gemini_model():
    model = ChatGoogleGenerativeAI(
        google_api_key=os.getenv("GOOGLE_API_KEY"),
        model="gemini-2.5-flash-lite",
        temperature=0,
    )
    return model

In [3]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback on the essay")
    score: float = Field(ge=0, le=10, description="Score between 0 and 10")
    
llm = get_gemini_model()
llm_structured = llm.with_structured_output(EvaluationSchema)

In [4]:
class UPSCState(TypedDict):
    essay_text: str
    clarity_feedback: str
    analysis_feedback: str
    language_feedback: str
    overall_feedback: str
    individual_score: Annotated[list[int], operator.add]
    final_score: float 

In [15]:
def evaluate_language(state: UPSCState) -> UPSCState:
    prompt = f"""
    You are an expert evaluator for UPSC essays. Evaluate the following essay for language clarity, grammar, and coherence. Provide detailed feedback and a score between 0 and 10.

    Essay:
    {state['essay_text']}
    """
    result = llm_structured.invoke(prompt)
    return {'language_feedback': result.feedback, 'individual_score': [result.score]}

def evaluate_thought(state: UPSCState) -> UPSCState:
    prompt = f"""
    You are an expert evaluator for UPSC essays. Evaluate the following essay for depth of thought, originality, and critical analysis. Provide detailed feedback and a score between 0 and 10.

    Essay:
    {state['essay_text']}
    """
    result = llm_structured.invoke(prompt)
    return {'clarity_feedback': result.feedback, 'individual_score': [result.score]}

def evaluate_analysis(state: UPSCState) -> UPSCState:
    prompt = f"""
    You are an expert evaluator for UPSC essays. Evaluate the following essay for analytical skills, use of evidence, and argument structure. Provide detailed feedback and a score between 0 and 10.

    Essay:
    {state['essay_text']}
    """
    result = llm_structured.invoke(prompt)
    return {'analysis_feedback': result.feedback, 'individual_score': [result.score]}

def final_evaluation(state: UPSCState) -> UPSCState:
    prompt = f"""
    You are an expert evaluator for UPSC essays. Based on the following feedback from different evaluators, provide an overall feedback summary.

    Language Feedback:
    {state['language_feedback']}

    Thought Feedback:
    {state['clarity_feedback']}

    Analysis Feedback:
    {state['analysis_feedback']}
    """
    result = llm.invoke(prompt)
    overall_feedback = result.content
    final_score = sum(state['individual_score']) / len(state['individual_score'])
    return {'overall_feedback': overall_feedback, 'final_score': final_score}

In [16]:
graph = StateGraph(UPSCState)

#nodes
graph.add_node("Evaluate Language", evaluate_language)
graph.add_node("Evaluate Thought", evaluate_thought)
graph.add_node("Evaluate Analysis", evaluate_analysis)
graph.add_node("Final Evaluation", final_evaluation)

#edges
graph.add_edge(START, "Evaluate Language")
graph.add_edge(START, "Evaluate Thought")
graph.add_edge(START, "Evaluate Analysis")
graph.add_edge("Evaluate Language", "Final Evaluation")
graph.add_edge("Evaluate Thought", "Final Evaluation")
graph.add_edge("Evaluate Analysis", "Final Evaluation")
graph.add_edge("Final Evaluation", END)

workflow = graph.compile()

In [17]:
initial_state = {
    "essay_text": """Every generation is defined by unique values, priorities, and approaches to life. With the rise of Gen Z—those born roughly between 1997 and 2012—the workplace has witnessed fresh energy, creativity, and technological savvy. However, along with these strengths, a recurring criticism is that Gen Zs appear irresponsible when it comes to fulfilling their responsibilities towards work. While this generalization does not apply to every individual, there are noticeable patterns that raise concerns among employers and older colleagues.

One of the most prominent issues is the lack of commitment. Many Gen Z employees view jobs as stepping stones rather than long-term careers. Instead of dedicating themselves fully to one role, they tend to switch frequently in search of higher pay, flexible hours, or a more fulfilling environment. While job mobility is not inherently negative, frequent shifts can reflect a lack of accountability and disrupt organizational stability.

Secondly, work ethic and patience are often questioned. Gen Z has grown up in a fast-paced, instant-gratification world shaped by social media, apps, and quick solutions. As a result, they may expect rapid promotions, recognition, and rewards without putting in the sustained effort traditionally required. When challenges arise, some are quick to disengage or complain rather than demonstrate resilience and responsibility.

Another factor is their blurred boundary between personal life and professional duties. Gen Z values work-life balance to such an extent that they often prioritize personal well-being, hobbies, or side hustles over the core responsibilities of their main job. While self-care is important, the excessive focus on personal comfort sometimes comes at the expense of organizational goals and deadlines.

Moreover, Gen Z tends to exhibit overreliance on technology. While their digital fluency is a strength, it can also lead to irresponsibility. For instance, when faced with tasks requiring interpersonal communication, negotiation, or deep critical thinking, some Gen Z employees may struggle or avoid responsibility. Dependence on shortcuts and online tools may undermine accountability in situations that demand ownership and perseverance.

In conclusion, Gen Z brings innovation, creativity, and adaptability to the workplace, but their approach is often criticized as irresponsible with regard to work responsibilities. Their frequent job-switching, impatience for growth, emphasis on personal over professional duties, and overreliance on technology highlight challenges for employers. To bridge this gap, organizations must create environments that balance Gen Z’s need for flexibility with the demands of accountability, while Gen Z must cultivate stronger discipline and long-term responsibility. Only then can they overcome this perception and contribute sustainably to the future of work."""
}

In [18]:
workflow.invoke(initial_state)

{'essay_text': 'Every generation is defined by unique values, priorities, and approaches to life. With the rise of Gen Z—those born roughly between 1997 and 2012—the workplace has witnessed fresh energy, creativity, and technological savvy. However, along with these strengths, a recurring criticism is that Gen Zs appear irresponsible when it comes to fulfilling their responsibilities towards work. While this generalization does not apply to every individual, there are noticeable patterns that raise concerns among employers and older colleagues.\n\nOne of the most prominent issues is the lack of commitment. Many Gen Z employees view jobs as stepping stones rather than long-term careers. Instead of dedicating themselves fully to one role, they tend to switch frequently in search of higher pay, flexible hours, or a more fulfilling environment. While job mobility is not inherently negative, frequent shifts can reflect a lack of accountability and disrupt organizational stability.\n\nSecond